<img width="8%" alt="Snowflake.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Snowflake.png" style="border-radius: 15%">

# Snowflake - Basics and data querying
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Snowflake+-+Basics+and+data+querying:+Error+short+description">Bug report</a>

**Tags:** #snowflake #data #warehouse #naas_drivers #snippet

**Author:** [Mateusz Polakowski](https://www.linkedin.com/in/polakowski/)

**Last update:** 2023-04-12 (Created: 2022-08-06)

**Description:** This notebook provides an introduction to the basics of Snowflake and how to query data within it.

## Input

### Import library

In [1]:
import os
from naas_drivers import snowflake
from snowflake.connector.errors import ProgrammingError

### Setup Snowflake account

If you don't have your SF account, you can easily set up a [30-day trial account with $400 budget here](https://signup.snowflake.com/).

To get your Snowflake account ID essential for connecting, please refer to [Account Identifiers in Snowflake documentation](https://docs.snowflake.com/en/user-guide/admin-account-identifier.html). There are several methods to get your account ID, but the overall rule can be found below:

```<account_identifier>.snowflakecomputing.com```

If you're proceeding with the trial account, it's highly probable that your ID will resemble something like: `xy1234.eu-central-1`.

### Credentials

In [2]:
# Here environment variables are used to pass Snowflake credentials,
# but it's okay to do it in a different manner

sf_username = os.environ["SNOWFLAKE_USER"]
sf_password = os.environ["SNOWFLAKE_PASSWORD"]
sf_account = os.environ["SNOWFLAKE_ACCOUNT"]

## Model

### Connecting to your Snowflake account

In [3]:
snowflake.connect(username=sf_username, password=sf_password, account=sf_account)

### Environment setup

In [4]:
snowflake.database.get_current() is None

In [5]:
snowflake.set_environment(
    warehouse="COMPUTE_WH",
    database="SNOWFLAKE_SAMPLE_DATA",
    schema="TPCH_SF100",
    role="ACCOUNTADMIN",
)

In [6]:
snowflake.get_environment()

## Output

### Creating new database and schema

In [7]:
snowflake.database.create("NAAS", or_replace=True)

In [8]:
snowflake.database.use("NAAS", silent=True)

In [9]:
snowflake.schema.create("INGESTION_SCHEMA", silent=True)

In [10]:
snowflake.schema.use("INGESTION_SCHEMA", True)

### Executing custom query with a cursor

In [11]:
snowflake.cursor.execute("SHOW SCHEMAS;").fetchall()

### Data querying - wrong schema

In [12]:
snowflake.get_environment()

In [13]:
query = "SELECT * FROM CUSTOMER;"

In [14]:
# Querying table that doesn't exist in NAAS/INGESTION_SCHEMA
try:
    results_1_not_working = snowflake.execute(query)
except ProgrammingError as pe:
    print("Something went wrong!")
    print(pe)

### Data querying - valid command run with session environment properly set up

In [15]:
snowflake.set_environment(database="SNOWFLAKE_SAMPLE_DATA", schema="TPCH_SF100")

In [16]:
results_1_working = snowflake.execute(query)
results_1_working

### Data querying - another valid command run

In [17]:
results_2 = snowflake.execute(query, n=100)

print(f"Rows returned: {len(results_2['results'])}")
results_2["results"][:2]

### Data querying - mapping results to Pandas DataFrame

In [18]:
results_pandas = snowflake.query_pd(query, n=100)

print(f"Rows returned: {len(results_pandas)}")
results_pandas.head(2)

## Extra

### Objects: `cursor` and `connection`

Both provided by Snowflake connector, that allow to execute any functionality possible with the original connector.

In [19]:
snowflake.cursor.execute("SELECT CURRENT_WAREHOUSE()").fetchall()

In [20]:
snowflake.connection.database